# Public Transit Options Around A Location

---
Azure Maps Mobility service (http://bit.ly/332vWZv) allows functionality for public transit, bike share, and scooter share routing with multimodal trip planning options. For public transit, the service offers search for nearby stations, stops, and lines and provides information like scheduled and real-time public transit arrivals, list of stops and route guides, as well as detailed and multimodal step-by-step itineraries and service alerts. The mobility service also returns real-time and static availability information to car share vehicles and shared bikes and scooters. 

I have included a few scenarios in this walkthrough:

1. Geocoding to find the lat long for any address
2. Finding the MetroID for use in the Mobility API for the address you are analyzing
3. Using the Find Nearby Transit Options API

For Geocoding, Azure Maps does not have the same level of information and accuracy for all cities and countries/regions. Check this table to see if your region is covered: http://bit.ly/2LHMiAV


---

In [1]:
import os
import json
import time
import requests
import urllib.parse
from IPython.display import Image, display

Load Subscription Key for Azure Maps from Config File. If you dont have an Azure Maps Subscription Key, get yourself a free account here: http://bit.ly/2VV39SG

In [2]:
config_file = 'config.json'
key = json.load(open(config_file))['key']
subscriptionKey = key

The Azure Maps Search Service provides the ability for customers to find real world objects and their respective location. The Search Service provides for three major functions:
    1. Geocoding: Finding addresses, places, and landmarks
    2. Point of Interest (POI) Search: Finding businesses based on a location
    3. Reverse Geocoding: Finding addresses or cross streets based on a location
    
You can customize this for the address that you want analyze by replacing the address below:

In [3]:
addresstoAnalyze = "400, Pine Street, Seattle"
locationDetails = requests.get("https://atlas.microsoft.com/search/address/json?subscription-key={}&api-version=1.0&query={}".format(subscriptionKey,addresstoAnalyze)).json()
latlong=locationDetails["results"][0]["position"]
currentLocation=[0,0]
currentLocation[0]=latlong["lat"]
currentLocation[1]=latlong["lon"]

The *Get Metro Area* API allows to request metro areas in which the Azure Maps Mobility Service is available. The service supports filtering results by country or coordinate location. Information returned includes Metro Area details such as metro Id, name and a representation of the metro area geometry in GeoJSON format.

In [4]:
metroAreaID = requests.get("https://atlas.microsoft.com/mobility/metroArea/id/json?subscription-key={}&api-version=1.0&queryType=position&query={}".format(subscriptionKey,str(currentLocation[0])+","+str(currentLocation[1]))).json()
metroID1 = metroAreaID["results"][0]["metroId"]
metroName1 = metroAreaID["results"][0]["metroName"]
print("The Metro ID for", metroName1, "is",metroID1)

The Metro ID for Seattle–Tacoma–Bellevue, WA is 522


The Get Nearby Transit service (http://bit.ly/31Ui6Iw) allows you to search transit objects, for example, public transit stops and shared bikes around a given location returning the transit object details. The Service allows users to search for specific object types and within a given radius returning a set of transit object with object details.

In [5]:
transitOptions = requests.get("https://atlas.microsoft.com/mobility/transit/nearby/json?subscription-key={}&api-version=1.0&metroId={}&query={}&radius=1000".format(subscriptionKey,metroID1,str(currentLocation[0])+","+str(currentLocation[1]))).json()

This is the returned json from the Get Nearby Transit Service with the details of the nearby stops

In [6]:
transitOptionsF = json.dumps(transitOptions, indent=4)
print(transitOptionsF)

{
    "results": [
        {
            "id": "522---2055334",
            "type": "stop",
            "objectDetails": {
                "stopKey": "760",
                "stopName": "5th Ave & Pine St",
                "stopCode": "760",
                "mainTransitType": "Bus",
                "mainAgencyId": "522---5872",
                "mainAgencyName": "Metro Transit"
            },
            "position": {
                "latitude": 47.611946,
                "longitude": -122.336921
            },
            "viewport": {
                "topLeftPoint": {
                    "latitude": 47.612853,
                    "longitude": -122.33824
                },
                "btmRightPoint": {
                    "latitude": 47.611038,
                    "longitude": -122.335601
                }
            }
        },
        {
            "id": "522---2055357",
            "type": "stop",
            "objectDetails": {
                "stopKey": "1108",
              

We enumerate through the elements in the json results to list out all of the transit stops around the location that you are analyzing. In addition, for every stop, we are also calling the Transit Stop Info service (http://bit.ly/2ARnDn7) to get details of all the lines that go through that stop. The Get Transit Stop Info service allows you to request information for a given public transit stop. Basic information returned includes details such as main transit type of most lines stopping for a given public and main transport agency. Additional details such as stop lines, active service alerts for specified stop, photos and user reviews are also available, depending on the options selected.

In [7]:
print("Here are the public transit stops: \n")
totTime = 0 #used for calculating the average walk time to stops
totDist = 0 #used ro calculating the average distance to stops
for loc in range(len(transitOptions["results"])):
                #this string is the lat long for the location being analyzed and the lat long of the stops
                starttoDest = str(currentLocation[0])+","+str(currentLocation[1])+":"+ str(transitOptions["results"][loc]["position"]["latitude"])+","+str(transitOptions["results"][loc]["position"]["longitude"])
                tMode = "pedestrian" # we will measure the walking time & route to the stops
                # the Get Route Directions API (http://bit.ly/2oauGF5) returns a route between an origin and a destination, passing through waypoints if they are specified. 
                routeDetails = requests.get("https://atlas.microsoft.com/route/directions/json?subscription-key={}&api-version=1.0&query={}&travelMode={}".format(subscriptionKey,starttoDest,tMode)).json()
                # this is the walking time to the stop in minutes
                timeToStop = routeDetails["routes"][0]["summary"]["travelTimeInSeconds"]/60
                totTime = totTime + timeToStop
                # this is the distance in kms to the stop
                distToStop = routeDetails["routes"][0]["summary"]["lengthInMeters"]/1000
                totDist = totDist + distToStop
                print("\n",loc+1,". ",transitOptions["results"][loc]["objectDetails"]["mainTransitType"],"(", transitOptions["results"][loc]["objectDetails"]["mainAgencyName"],")", ":", transitOptions["results"][loc]["objectDetails"]["stopName"], ". Walk Time in Minutes:", timeToStop,". Distance in Km:", distToStop)
                sId = transitOptions["results"][loc]["id"]
                # The Get Transit Stop (http://bit.ly/2ARnDn7) Info service allows you to request information for a given public transit stop. Basic information returned includes details such as main transit type of most lines stopping for a given public and main transport agency. 
                stopDetails = requests.get("https://atlas.microsoft.com/mobility/transit/stop/json?subscription-key={}&api-version=1.0&metroId={}&query={}&queryType=stopId&detailType=lines".format(subscriptionKey,metroID1,sId)).json()
                print("\n\tLines:")
                for loc1 in range(len(stopDetails["lines"])):
                    print ("\t\t",loc1+1, ".", stopDetails["lines"][loc1]["lineDestination"], "-",stopDetails["lines"][loc1]["direction"])
print ("\nTotal Transit Stops Nearby:",len(transitOptions["results"])) 
print ("Average Distance to Stops in Kms",totDist/len(transitOptions["results"])) 
print ("Average Time Walk to Stops in Minutes",totTime/len(transitOptions["results"]))      

Here are the public transit stops: 


 1 .  Bus ( Metro Transit ) : 5th Ave & Pine St . Walk Time in Minutes: 2.2 . Distance in Km: 0.183

	Lines:
		 1 . Downtown Seattle Via Sr-520 - backward
		 2 . Downtown Seattle - backward
		 3 . Downtown Seattle Kirkland - backward
		 4 . Downtown Seattle - backward
		 5 . Downtown Seattle Via Sr-520 - backward
		 6 . Downtown Seattle Via Sr-520 - backward
		 7 . Downtown Seattle - backward

 2 .  Tram ( Sound Transit ) : Westlake Station - Bay C . Walk Time in Minutes: 0.6666666666666666 . Distance in Km: 0.055

	Lines:
		 1 . Angle Lake Station - forward

 3 .  Tram ( Seattle Monorail ) : Westlake Station . Walk Time in Minutes: 19.7 . Distance in Km: 1.641

	Lines:
		 1 . Seattle Center - backward
		 2 . Westlake Center - forward

Total Transit Stops Nearby: 3
Average Distance to Stops in Kms 0.6263333333333333
Average Time Walk to Stops in Minutes 7.522222222222222


The returned data can be used for calculating the Transit Score based on:
1. Number of Transit Stops within the specified Radius
2. Distance to the Transit Stops from your location
3. Number of lines that go through the Transit Stops
4. Destinations of each of the lines
5. Frequency & timings of the transit options
6. & more